## Get the Data

Either use the provided .csv file or (optionally) get fresh (the freshest?) data from running an SQL query on StackExchange: 

Follow this link to run the query from [StackExchange](https://data.stackexchange.com/stackoverflow/query/675441/popular-programming-languages-per-over-time-eversql-com) to get your own .csv file

<code>
select dateadd(month, datediff(month, 0, q.CreationDate), 0) m, TagName, count(*)
from PostTags pt
join Posts q on q.Id=pt.PostId
join Tags t on t.Id=pt.TagId
where TagName in ('java','c','c++','python','c#','javascript','assembly','php','perl','ruby','visual basic','swift','r','object-c','scratch','go','swift','delphi')
and q.CreationDate < dateadd(month, datediff(month, 0, getdate()), 0)
group by dateadd(month, datediff(month, 0, q.CreationDate), 0), TagName
order by dateadd(month, datediff(month, 0, q.CreationDate), 0)
</code>

## Import Statements

In [212]:
import pandas as pd

## Data Exploration

Read the .csv file and store it in a Pandas dataframe

In [213]:
df = pd.read_csv("QueryResults.csv", names=['DATE', 'TAG', 'POSTS']).dropna()

Examine the first 5 rows and the last 5 rows of the of the dataframe

In [214]:
df.head()

,DATE,TAG,POSTS
1,2008-07-01 00:00:00,c#,3.0
2,2008-08-01 00:00:00,ruby,70.0
3,2008-08-01 00:00:00,perl,28.0
4,2008-08-01 00:00:00,python,119.0
5,2008-08-01 00:00:00,delphi,13.0


In [215]:
df.tail()

,DATE,TAG,POSTS
2617,2024-04-01 00:00:00,go,363.0
2618,2024-04-01 00:00:00,r,1657.0
2619,2024-04-01 00:00:00,delphi,112.0
2620,2024-04-01 00:00:00,php,1487.0
2621,2024-04-01 00:00:00,ruby,124.0


Check how many rows and how many columns there are. 
What are the dimensions of the dataframe?

In [216]:
df.shape

(2621, 3)

Count the number of entries in each column of the dataframe

In [217]:
df.count()

DATE     2621
TAG      2621
POSTS    2621
dtype: int64

## Data Cleaning

Let's fix the date format to make it more readable. We need to use Pandas to change format from a string of "2008-07-01 00:00:00" to a datetime object with the format of "2008-07-01"

In [218]:
# Convert the 'DATE' column to datetime
df['DATE'] = pd.to_datetime(df['DATE'])

In [219]:
# Extract only the date part
df["DATE"] = df["DATE"].dt.date

Calculate the total number of post per language.
Which Programming language has had the highest total number of posts of all time?

In [220]:
top_searched_languages = df.groupby("TAG").sum("POSTS")
top_searched_languages.sort_values("POSTS", ascending=False)

,POSTS
TAG,
javascript,2529059.0
python,2194524.0
java,1917704.0
c#,1616186.0
php,1464810.0
c++,807228.0
r,506190.0
c,404310.0
swift,333610.0


Some languages are older (e.g., C) and other languages are newer (e.g., Swift). The dataset starts in September 2008.

**Challenge**: How many months of data exist per language? Which language had the fewest months with an entry? 


## Data Manipulation



**Challenge**: What are the dimensions of our new dataframe? How many rows and columns does it have? Print out the column names and print out the first 5 rows of the dataframe.

**Challenge**: Count the number of entries per programming language. Why might the number of entries be different? 

## Data Visualisaton with with Matplotlib


**Challenge**: Use the [matplotlib documentation](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.plot.html#matplotlib.pyplot.plot) to plot a single programming language (e.g., java) on a chart.

**Challenge**: Show two line (e.g. for Java and Python) on the same chart.

# Smoothing out Time Series Data

Time series data can be quite noisy, with a lot of up and down spikes. To better see a trend we can plot an average of, say 6 or 12 observations. This is called the rolling mean. We calculate the average in a window of time and move it forward by one overservation. Pandas has two handy methods already built in to work this out: [rolling()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) and [mean()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.window.rolling.Rolling.mean.html). 